# Using Keras RL

In [13]:
import gym_fishing
import gym
import math
import random
from math import floor
import bokeh.plotting
import numpy as np
import bokeh.io
from itertools import count
import bokeh.plotting
from bokeh.models import Span
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from rl.memory import SequentialMemory
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy

bokeh.io.output_notebook()

Loading BokehJS ...

For proof of concept, I am going to use the DQN RL agent for the CartPole environment.

In [14]:
# Instatiating the environment then making a step to show the output
cartpole_env = gym.make('CartPole-v0')
cartpole_env.reset()
cartpole_env.step(0)

(array([ 0.04493336, -0.21803191, -0.01117   ,  0.28806079]), 1.0, False, {})

Now I make a FF neural net with 2 hidden layers each wih width 128 neurons.

In [15]:
def agent(shape, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, shape)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

Now, training the agent.

In [16]:
model = agent(4, 2)
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannGumbelQPolicy()

dqn = DQNAgent(model=model, policy=policy, nb_actions=2, memory=memory)
dqn.compile('adam', metrics = ['mae'])
dqn.fit(cartpole_env, nb_steps = 1000, visualize = False, verbose = 1)

Training for 1000 steps ...
Interval 1 (0 steps performed)
  978/10000 [=>............................] - ETA: 6s - reward: 1.0000done, took 0.702 seconds


Testing the agent

In [17]:
_ = dqn.test(cartpole_env, nb_episodes = 5, visualize= False)

Testing for 5 episodes ...
Episode 1: reward: 9.000, steps: 9
Episode 2: reward: 9.000, steps: 9
Episode 3: reward: 10.000, steps: 10
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 9.000, steps: 9


This all looks good.

Now, doing the same for the fishing gym environment, I first instatiate the gym.

In [18]:
gym_name = 'fishing-v0'

In [19]:
# Instatiating the environment then making a step to show the output
env = gym.make(gym_name)
env.reset()
env.step(0)

(array([99802]), 0.0, False, {})

In [20]:
def agent(shape, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, shape)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

Here, I train the DQN agent. Only changing the `input_shape` from above.

In [21]:
model = agent(1, len(env.action_space))
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannGumbelQPolicy()

dqn = DQNAgent(model=model, policy=policy, nb_actions=len(env.action_space), memory=memory)
dqn.compile('adam', metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_3_input to have shape (1, 1) but got array with shape (1, 4)

In [22]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [23]:
%watermark -v -p numpy,jupyterlab,tensorflow,keras

CPython 3.7.4
IPython 7.8.0

numpy 1.15.4
jupyterlab 1.1.4
tensorflow 2.0.0-beta1
keras 2.3.1
